<a href="https://colab.research.google.com/github/julwdo/thesis/blob/main/01_codes/01_gcolab/01_thesis_preprocess_jsonl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import joblib

pd.set_option('display.max_columns', None)

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
path = "/content/drive/MyDrive/twibot-22/raw"

In [6]:
user = pd.read_json(f"{path}/user.jsonl", lines=True)

user = pd.concat([
    user,
    pd.json_normalize(user["entities"]),
    pd.json_normalize(user["public_metrics"])
    ], axis=1)

user.drop(columns=["entities", "public_metrics"], inplace=True)
user.head(1)

,created_at,description,id,location,name,pinned_tweet_id,profile_image_url,protected,url,username,verified,withheld,url.urls,description.urls,description.mentions,description.hashtags,description.cashtags,followers_count,following_count,tweet_count,listed_count
0,2020-01-16 02:02:55+00:00,Theoretical Computer Scientist. See also https...,u1217628182611927040,"Cambridge, MA",Boaz Barak,NaN,https://pbs.twimg.com/profile_images/125226236...,False,https://t.co/BoMip9FF17,boazbaraktcs,False,None,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...","[{'start': 41, 'end': 64, 'url': 'https://t.co...",NaN,NaN,NaN,7316,215,3098,69


In [7]:
joblib.dump(user, f"{path}/user.joblib")

['/content/drive/MyDrive/twibot-22/raw/user.joblib']

In [3]:
!apt-get install openjdk-17-jdk-headless -qq > /dev/null # OpenJDK 17
!wget --show-progress https://archive.apache.org/dist/spark/spark-3.5.6/spark-3.5.6-bin-hadoop3.tgz # Apache Spark 3.5.6 with Hadoop 3 support
!tar xf spark-3.5.6-bin-hadoop3.tgz
!pip install -q findspark

--2026-02-06 18:12:21--  https://archive.apache.org/dist/spark/spark-3.5.6/spark-3.5.6-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400923510 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.6-bin-hadoop3.tgz’

spark-3.5.6-bin-had 100%[===================>] 382.35M   371KB/s    in 15m 59s 

2026-02-06 18:28:22 (408 KB/s) - ‘spark-3.5.6-bin-hadoop3.tgz’ saved [400923510/400923510]



In [4]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-3.5.6-bin-hadoop3'

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [8]:
from pyspark.sql import functions as F

tweet_0 = (
    spark.read.json(f"{path}/tweet_0.jsonl")
    .select(
        "*",
        F.col("entities.*"),
        F.col("public_metrics.*")
    )
)

tweet_0 = tweet_0.drop("entities", "public_metrics")

In [10]:
tweet_0.write.mode("overwrite").parquet(f"{path}/tweet_0.parquet")